# Using 'open virtual dataset' capability to work with TEMPO Level 3 data

---

## Summary

In this tutorial, we will use the `earthaccess.open_virtual_mfdataset()` function to open a week's worth of granules from the Nitrogen Dioxide (NO<sub>2</sub>) Level-3 data collection of the [TEMPO air quality mission](https://asdc.larc.nasa.gov/project/TEMPO).

**About TEMPO**: The Tropospheric Emissions: Monitoring of Pollution (TEMPO) instrument is a geostationary satellite mission that provides hourly daytime measurements of air quality over North America. It measures key pollutants including nitrogen dioxide (NO<sub>2</sub>), formaldehyde, and ozone at high spatial resolution (~2 by 4.75 km at the center of its field of regard).

We will calculate temporal and spatial means for a subset of the data and visualize the results. This approach demonstrates cloud-optimized data access patterns that can scale from days to years of data.

**Learn more**: For comprehensive documentation on the `earthaccess` package, visit the [earthaccess documentation](https://earthaccess.readthedocs.io/).

Note that this same approach can be used for a date range of any length, within the mission's duration. Running this notebook for a year's worth of TEMPO Level-3 data took approximately 15 minutes.

## Prerequisites

- **AWS US-West-2 Environment:** This tutorial has been designed to run in an AWS cloud compute instance in AWS region us-west-2. However, if you want to run it from your laptop or workstation, everything should work just fine but without the speed benefits of in-cloud access.

- **Earthdata Account:** A (free!) Earthdata Login account is required to access data from the NASA Earthdata system. Before requesting TEMPO data, we first need to set up our Earthdata Login authentication, as described in the Earthdata Cookbook's [earthaccess tutorial (link)](https://nasa-openscapes.github.io/earthdata-cloud-cookbook/tutorials/earthaccess-demo.html).

- **Packages:**

  - `cartopy`
  - `dask`
  - `earthaccess` version 0.14.0 or greater
  - `matplotlib`
  - `numpy`
  - `xarray`
## Setup

In [ ]:
import cartopy.crs as ccrs
import earthaccess
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
from matplotlib import rcParams

%config InlineBackend.figure_format = 'jpeg'
rcParams["figure.dpi"] = (
    80  # Reduce figure resolution to keep the saved size of this notebook low.
)

## Login using the Earthdata Login

In [ ]:
auth = earthaccess.login()

if not auth.authenticated:
    # Ask for credentials and persist them in a .netrc file.
    auth.login(strategy="interactive", persist=True)

print(earthaccess.__version__)

## Search for data granules

We search for TEMPO Nitrogen Dioxide (NO<sub>2</sub>) data for a week-long period (note: times are in UTC), between January 11th and 18th, 2024.
We are limiting the results to render the notebook quickly.

In [ ]:
results = earthaccess.search_data(
    # TEMPO NO₂ Level-3 product
    short_name="TEMPO_NO2_L3",
    # Version 3 of the data product
    version="V03",
    # Time period: One week in January 2024 (times are in UTC)
    temporal=("2024-01-11 12:00", "2024-01-18 12:00"),
    count=8,
)

print(f"Number of granules found: {len(results)}")

## Opening Virtual Multifile Datasets

## Understanding Virtual Datasets

Virtual datasets allow us to work with multiple files as if they were a single dataset without downloading all the data to local storage. This is achieved through:

1. **Kerchunk**: Creates lightweight reference files that point to data chunks in cloud storage
2. **Virtualizarr**: Combines multiple reference files into a single virtual dataset
3. **Lazy Loading**: Data is only accessed when needed for computations

For TEMPO data, we need to handle the hierarchical netCDF4 structure by opening each group separately, then merging them.

First we set the argument options to be used by `earthaccess.open_virtual_mfdataset`.

<div class="alert alert-block alert-info">

`load` argument considerations:

- `load=True` works.  Within `earthaccess.open_virtual_mfdataset`, a temporary virtual reference file (a "virtual dataset") is created and then immediately loaded with kerchunk. This is because the function assumes the user is making this request for the first time and the combined manifest file needs to be generated first. In the future, however, `earthaccess.open_virtual_mfdataset` may provide a way to save the combined manifest file, at which point you could then avoid repeating these steps, and proceed directly to loading with kerchunk/virtualizarr.
- `load=False` results in `KeyError: "no index found for coordinate 'longitude'"` because it creates `ManifestArray`s without indexes (see the [earthaccess documentation here (link)](https://github.com/nsidc/earthaccess/blob/7f5fe5d2e42343b6d7948338255cf9bb8cdb2775/earthaccess/dmrpp_zarr.py#L36C456-L36C502))
</div>

In [ ]:
open_options = {
    "access": "indirect",  # access to cloud data (faster in AWS with "direct")
    "load": True,  # Load metadata immediately (required for indexing)
    "concat_dim": "time",  # Concatenate files along the time dimension
    "data_vars": "minimal",  # Only load data variables that include the concat_dim
    "coords": "minimal",  # Only load coordinate variables that include the concat_dim
    "compat": "override",  # Avoid coordinate conflicts by picking the first
    "combine_attrs": "override",  # Avoid attribute conflicts by picking the first
}

Because TEMPO data are processed and archived in a netCDF4 format using a group hierarchy,
we open each group – i.e., 'root', 'product', and 'geolocation' – and then afterwards merge them together.

In [ ]:
%%time
result_root = earthaccess.open_virtual_mfdataset(granules=results, **open_options)
result_product = earthaccess.open_virtual_mfdataset(
    granules=results, group="product", **open_options
)
result_geolocation = earthaccess.open_virtual_mfdataset(
    granules=results, group="geolocation", **open_options
)
# merge
result_merged = xr.merge([result_root, result_product, result_geolocation])
result_merged

In [ ]:
result_merged = xr.merge([result_root, result_product, result_geolocation])
result_merged

## Understanding the Data

- **vertical_column_troposphere**: Total column amount of NO₂ in the troposphere (units: molecules/cm²)
- **main_data_quality_flag**: Quality indicator (0 = good quality data)
- **Geographic region**: We'll focus on the Mid-Atlantic region (Washington DC area)
  - Longitude: -78° to -74° W
  - Latitude: 35° to 39° N

In [ ]:
# Define our region of interest (Mid-Atlantic/Washington DC area)
lon_bounds = (-78, -74)  # Western to Eastern longitude
lat_bounds = (35, 39)  # Southern to Northern latitude

print(
    f"Analyzing region: {lat_bounds[0]}°N to {lat_bounds[1]}°N, {abs(lon_bounds[0])}°W to {abs(lon_bounds[1])}°W"
)

## Temporal Mean - a map showing an annual average

In [ ]:
# Define temporal mean (average over time) calculation
temporal_mean_ds = (
    result_merged.sel(
        {
            "longitude": slice(lon_bounds[0], lon_bounds[1]),
            "latitude": slice(lat_bounds[0], lat_bounds[1]),
        }
    )
    .where(result_merged["main_data_quality_flag"] == 0)  # Filter for good quality data
    .mean(dim="time")
)

print(f"Dataset shape after subsetting: {temporal_mean_ds.dims}")
temporal_mean_ds

In [ ]:
%%time
# Compute the temporal mean
mean_vertical_column_trop = temporal_mean_ds["vertical_column_troposphere"].compute()

In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection": ccrs.PlateCarree()})

mean_vertical_column_trop.squeeze().plot.contourf(ax=ax)

# Add geographic features
ax.coastlines()
ax.gridlines(
    draw_labels=True,
    dms=True,
    x_inline=False,
    y_inline=False,
)
plt.show()

## Spatial mean - a time series of area averages

In [ ]:
# Define spatial mean (average over longitude/latitude) calculation
spatial_mean_ds = (
    result_merged.sel(
        {
            "longitude": slice(lon_bounds[0], lon_bounds[1]),
            "latitude": slice(lat_bounds[0], lat_bounds[1]),
        }
    )
    .where(result_merged["main_data_quality_flag"] == 0)  # Filter for good quality data
    .mean(dim=("longitude", "latitude"))
)
spatial_mean_ds

In [ ]:
%%time
# Compute the spatial mean
spatial_mean_vertical_column_trop = spatial_mean_ds[
    "vertical_column_troposphere"
].compute()

In [ ]:
spatial_mean_vertical_column_trop.plot()
plt.show()

## Single scan subset

In [ ]:
# Select a single scan time for detailed analysis
scan_time_start = np.datetime64("2024-01-11T13:00:00")  # 1 PM UTC
scan_time_end = np.datetime64("2024-01-11T14:00:00")  # 2 PM UTC

print(f"Analyzing single scan: {scan_time_start} to {scan_time_end} UTC")
print("Note: This corresponds to ~8-9 AM local time on the US East Coast")

subset_ds = result_merged.sel(
    {
        "longitude": slice(lon_bounds[0], lon_bounds[1]),
        "latitude": slice(lat_bounds[0], lat_bounds[1]),
        "time": slice(scan_time_start, scan_time_end),
    }
).where(result_merged["main_data_quality_flag"] == 0)
subset_ds

In [ ]:
%%time
# Compute the single scan's values
subset_vertical_column_trop = subset_ds["vertical_column_troposphere"].compute()

In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection": ccrs.PlateCarree()})

subset_vertical_column_trop.squeeze().plot.contourf(ax=ax)

# Add geographic features
ax.coastlines()
ax.gridlines(
    draw_labels=True,
    dms=True,
    x_inline=False,
    y_inline=False,
)
plt.show()